In [1]:
import numpy as np
import tensorflow as tf
import pretty_midi
import pathlib
import glob

class notenClass:
    def __init__(self, toonHoogte, beginTijd, eindTijd, nootInterval, nootTijd):
        self.toonHoogte = toonHoogte
        self.beginTijd = beginTijd
        self.eindTijd = eindTijd
        self.nootInterval = nootInterval
        self.nootTijd = nootTijd

# Zet MIDI bestand om in een array van noten met hun bijhorende eigenschappen (bijvoorbeeld: toonhoogte).
def midiNaarNoten(bestand):
    noten = notenClass([], [], [], [], [])
    piano = pretty_midi.PrettyMIDI(bestand).instruments[0]

    # Rangschik alle gespeelde noten door de piano op chronologische volgorde.
    notenGerangschikt = sorted(piano.notes, key = (lambda noot : noot.start))

    # Orden alle noten met haar eigenschappen in de noten class.
    beginVorigeNoot = notenGerangschikt[0].start
    for noot in notenGerangschikt:
        noten.toonHoogte.append(noot.pitch)
        noten.beginTijd.append(noot.start)
        noten.eindTijd.append(noot.end)
        noten.nootInterval.append(noot.start - beginVorigeNoot)
        noten.nootTijd.append(noot.end - noot.start)
        beginVorigeNoot = noot.start
    
    # Creeër een array 
    notenArray = np.stack((noten.toonHoogte, noten.beginTijd, noten.eindTijd, noten.nootInterval, noten.nootTijd), axis = 1)
    
    return notenArray

def normalizeerToonHoogte(notenArray):
    notenArray = notenArray/[maxToonHoogte, 1.0, 1.0]
    return notenArray

# Zet een array van noten met haar bijhorende eigenschappen om in een MIDI bestand.
def notenNaarMidi(noten, exportBestand, instrumentNaam = 'Acoustic Grand Piano', geluidsterkte = 100):
    bestand = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=pretty_midi.instrument_name_to_program(instrumentNaam))

    # Zet de array van noten om in Pretty_MIDI noten en voeg deze toe aan het gegeven instrument.
    startVorigeNoot = 0
    for noot in noten:
        nootInterval = noot[3]
        nootTijd = noot[4]
        beginTijd = float(startVorigeNoot + nootInterval)
        eindTijd = float(beginTijd + nootTijd)
        noot = pretty_midi.Note(velocity = geluidsterkte, pitch = int(noot[0]), start = beginTijd, end = eindTijd)
        instrument.notes.append(noot)
        startVorigeNoot = beginTijd

    bestand.instruments.append(instrument)

    # Exporteer de Pretty_MIDI object naar een .midi/.mid bestand in de directory van dit programma.
    bestand.write(exportBestand)
    return bestand

# Deelt dataset op in batches met labels en genormalizeerde inputs.
def verwerkDataset(dataset, batchInputGrootte):
    batchInputGrootte += 1

    # Maak een nieuwe dataset die windows bevat van de dataset uit de input.
    windows = dataset.window(batchInputGrootte, shift=1, drop_remainder=True)

    # Zet de windows om in batches en 'flatten' deze batches tot een nieuwe dataset.
    flatten = lambda x: x.batch(batchInputGrootte, drop_remainder=True)
    flattenedDataset = windows.flat_map(flatten)

    # Deel batch op in inputs en een bijbehorend label. En normalizeer de inputs. 
    def deelNormalizeerBatches(notenArray):
        inputs = notenArray[:-1]
        labels = notenArray[-1]
        labelsDictionary = {}
        labelsDictionary.update({'toonHoogte': labels[0]})
        labelsDictionary.update({'nootInterval': labels[1]})
        labelsDictionary.update({'nootTijd': labels[2]})

        return normalizeerToonHoogte(inputs), labelsDictionary

    # Pas bovenstaande functie toe tot elke batch in de dataset.
    # 2e argument in Dataset.map(), laat deze bovenstaande functie parallel toegepast worden door CPU.
    notenDataset = flattenedDataset.map(deelNormalizeerBatches, num_parallel_calls=tf.data.AUTOTUNE)

    return notenDataset

# Dit is de loss-function. De meanSquaredError is een veelgebruikte loss-function binnen Deep Learning.
# Bij een grote verschil tussen de waarde en de verwachte waarde, wordt de 'fout' groot.
# Een klein verschil resulteert tot een kleine 'fout'.
def lossFunction(waarde, verwachteWaarde):
    meanSquaredError = (waarde - verwachteWaarde) ** 2

    # Maakt de waarde positief.
    positieveWaarde = tf.maximum(-verwachteWaarde, 0.0) * 10

    return tf.reduce_mean(meanSquaredError + positieveWaarde)

def maakNieuweNoot(noten, keras_model, temperature = 1.0):
    inputNoten = tf.expand_dims(noten, 0)
    gemaakteNoten = model.predict(inputNoten)
    preToonHoogte = gemaakteNoten['toonHoogte']
    nootInterval = gemaakteNoten['nootInterval']
    nootTijd = gemaakteNoten['nootTijd']
    preToonHoogte /= temperature
    toonHoogte = tf.random.categorical(preToonHoogte, num_samples=1)
    toonHoogte = tf.squeeze(toonHoogte, axis=-1)
    nootTijd = tf.squeeze(nootTijd, axis=-1)
    nootInterval = tf.squeeze(nootInterval, axis=-1)
    nootInterval = tf.maximum(0, nootInterval)
    nootTijd = tf.maximum(0, nootTijd)

    return int(toonHoogte), float(nootInterval), float(nootTijd)


maxToonHoogte = 128

2022-03-20 18:31:11.733788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-20 18:31:11.733831: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Directory van onze dataset.
directory = pathlib.Path('data/maestro-v2.0.0')

# Bestanden binnen onze dataset.
bestanden = []
for i in glob.glob('data/maestro-v2.0.0/*/*[.mid]*'):
    bestanden.append(i)

bestanden = glob.glob(str(directory/'**/*.midi*'))

aantalBestanden = 5
alleNoten = []

# Orden alle noten met haar eigenschappen van het gebruikte aantal muziekstukken in een array.
for bestand in bestanden[:aantalBestanden]:
    noten = midiNaarNoten(bestand)
    alleNoten.append(noten)

notenArray = np.concatenate(alleNoten)
nAlleNoten = len(notenArray)

# Verwijder de begin- en eindtijden voor alle noten.
trainingsNoten = np.delete(notenArray, [1, 2], 1)

# Zet de array van trainingsnoten om in een dataset voor Tensorflow.
notenDataset = tf.data.Dataset.from_tensor_slices(trainingsNoten)

# Hoeveelheid inputs in de batches.
batchInputGrootte = 25

verwerkteDataset = verwerkDataset(notenDataset, batchInputGrootte)

batchGrootte = 64
bufferGrootte = nAlleNoten - batchInputGrootte
trainingDataset = verwerkteDataset.shuffle(bufferGrootte).batch(batchGrootte, drop_remainder=True).cache().prefetch(tf.data.experimental.AUTOTUNE)

input_shape = (batchInputGrootte, 3)
learning_rate = 0.005

inputs = tf.keras.Input(input_shape)
x = tf.keras.layers.LSTM(128)(inputs)

outputs = {
              'toonHoogte': tf.keras.layers.Dense(128, name = 'toonHoogte')(x),
              'nootInterval': tf.keras.layers.Dense(1, name = 'nootInterval')(x),
              'nootTijd': tf.keras.layers.Dense(1, name = 'nootTijd')(x),
          }

model = tf.keras.Model(inputs, outputs)

loss = {
            'toonHoogte': tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits = True),
            'nootInterval': lossFunction,
            'nootTijd': lossFunction,
        }

optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

model.compile(loss = loss, optimizer = optimizer)

model.summary()

model.evaluate(trainingDataset, return_dict = True)

model.compile(loss = loss, loss_weights={'toonHoogte': 0.05, 'nootInterval': 1.0, 'nootTijd': 1.0}, optimizer =  optimizer)

model.evaluate(trainingDataset, return_dict = True)

callbacks = [
                tf.keras.callbacks.ModelCheckpoint(filepath = './training_checkpoints/ckpt_{epoch}', save_weights_only = True),
                tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 5, verbose = 1, restore_best_weights = True)
            ]

epochs = 1
history = model.fit(trainingDataset, epochs = epochs, callbacks = callbacks)

temperature = 2.0
nNieuweNoten = 120
testNoten = midiNaarNoten(bestanden[1])
gecorrigeerdeTestNoten = np.delete(testNoten, [1, 2], 1)

nieuweInput = normalizeerToonHoogte(gecorrigeerdeTestNoten[:batchInputGrootte])

gemaakteNoten = []
beginVorigeNoot = 0

# 
for i in range(nNieuweNoten):
  toonHoogte, nootInterval, nootTijd = maakNieuweNoot(nieuweInput, model, temperature)
  beginTijd = beginVorigeNoot + nootInterval
  eindTijd = beginVorigeNoot + nootTijd
  gemaakteNoot = (toonHoogte, nootInterval, nootTijd)
  gemaakteNoten.append((*gemaakteNoot, beginTijd, eindTijd))
  nieuweInput = np.delete(nieuweInput, 0, axis=0)
  nieuweInput = np.append(nieuweInput, np.expand_dims(gemaakteNoot, 0), axis=0)
  beginVorigeNoot = beginTijd

nieuweNoten = np.array(gemaakteNoten)

# Exporteer het nieuwe muziekstuk naar een MIDI file in de directory van dit programma.
exportBestand = 'test.midi'
exportMidi = notenNaarMidi(
    nieuweNoten, exportBestand)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 25, 3)]      0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 128)          67584       ['input_2[0][0]']                
                                                                                                  
 nootInterval (Dense)           (None, 1)            129         ['lstm_1[0][0]']                 
                                                                                                  
 nootTijd (Dense)               (None, 1)            129         ['lstm_1[0][0]']                 
                                                                                              

2022-03-20 18:32:07.437838: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 49145 of 58435


      7/Unknown - 14s 17ms/step - loss: 5.3367 - nootInterval_loss: 0.1002 - nootTijd_loss: 0.3871 - toonHoogte_loss: 4.8494

2022-03-20 18:32:08.951535: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


913/913 [==============================] - 40s 41ms/step - loss: 0.4247 - nootInterval_loss: 0.1034 - nootTijd_loss: 0.1158 - toonHoogte_loss: 4.1083
